In [1]:
import pandas as pd
from aerobot.io import load_training_data, load_validation_data
from aerobot.utls import process_data
from aerobot.models import LogisticClassifier, GeneralClassifier
import seaborn as sns

# A model based only on oxidation state features

The oxidation state of DSDNA is an analytic function of its GC content. For proteins and RNA, we actually need to calculate these values, so we load a reference table. 

The code below loads amino acid and DNA sequence content for each genome, calculates the nominal oxidation states of those, and then uses them as features in a 3 class predictor of oxygen utilization. 

TODO: need to get access to coding sequences to calculate ZC of RNA coding seqs. Josh is on this. 

In [2]:
CANONICAL_NTS = ['A', 'C', 'G', 'T']
GC_NTS = ['G', 'C']

def calc_gc(nt1_features_df):
    """Calculate the GC content of the canonical nucleotides.

    Args:
        nt1_features_df (pd.DataFrame): A dataframe containing the counts of
            single nucleotides.

    Returns:
        pd.Series: A series containing the GC content of each genome.
    """
    canonical_data = nt1_features_df[CANONICAL_NTS]
    gc = canonical_data[GC_NTS].sum(axis=1) / canonical_data.sum(axis=1)
    gc.name = 'gc_content'
    return gc


AA_NOSC_DF = pd.read_csv('aa_nosc.csv', index_col=0)
AA_NC = AA_NOSC_DF.NC
AA_ZC = AA_NOSC_DF.NOSC
CANONICAL_AAS = AA_NOSC_DF.index.tolist()

def calc_aa_zc(aa1_features_df):
    """Calculate the formal C oxidation state.

    Args:
        aa1_features_df (pd.DataFrame): A dataframe containing the counts of
            single amino acids.

    Returns:
        pd.Series: The formal C oxidation state.
    """
    aas = sorted(set(aa1_features_df.columns).intersection(CANONICAL_AAS))
    canonical_data = aa1_features_df[aas]
    NC_total = (canonical_data @ AA_NC[aas])
    Ne_total = (canonical_data @ (AA_ZC[aas] * AA_NC[aas]))
    mean_cds_zc = Ne_total / NC_total
    mean_cds_zc.name = 'cds_aa_zc'

    cols = [mean_cds_zc]
    for elt in 'CNOS':
        col = 'N{0}'.format(elt)
        ns = AA_NOSC_DF[col]
        totals = (canonical_data @ ns[aas])
        means = totals / canonical_data.sum(axis=1)
        means.name = 'cds_aa_{0}'.format(col)
        cols.append(means)
    return pd.concat(cols, axis=1)


NT_NOSC_DF = pd.read_csv('nt_nosc.csv')
RNA_NOSC_DF = NT_NOSC_DF[NT_NOSC_DF.type == 'RNA']
RNA_NOSC_DF = RNA_NOSC_DF.set_index('letter_code')
RNA_NC = RNA_NOSC_DF.NC
RNA_ZC = RNA_NOSC_DF.NOSC
# Includes DNA and RNA names
CANONICAL_NTS_ALL = RNA_NOSC_DF.index.unique().tolist()

def calc_rna_zc(nt1_features_df):
    """Calculate the formal C oxidation state of an RNA.

    Args:
        nt1_features_df (pd.DataFrame): A dataframe containing the counts of
            single nucleotides, either RNA or DNA. Assumed to single stranded.

    Returns:
        pd.Series: The formal C oxidation state of the RNA coding sequences.
    """
    my_nts = sorted(set(nt1_features_df.columns).intersection(CANONICAL_NTS_ALL))
    canonical_data = nt1_features_df[my_nts]
    NC_total = (canonical_data @ RNA_NC[my_nts])
    Ne_total = (canonical_data @ (RNA_ZC[my_nts] * RNA_NC[my_nts]))
    mean_cds_zc = Ne_total / NC_total
    mean_cds_zc.name = 'cds_nt_zc'

    cols = [mean_cds_zc]
    for elt in 'CNOS':
        col = 'N{0}'.format(elt)
        ns = RNA_NOSC_DF[col]
        totals = (canonical_data @ ns[my_nts])
        means = totals / canonical_data.sum(axis=1)
        means.name = 'cds_nt_{0}'.format(col)
        cols.append(means)
    return pd.concat(cols, axis=1)

In [3]:
print("Loading metadata...")

feature_type = "metadata"
training_data_metadata = load_training_data(feature_type=feature_type)
validation_data_metadata = load_validation_data(feature_type=feature_type)

train_ngenes = training_data_metadata["features"]["number_of_genes"]
test_ngenes = validation_data_metadata["features"]["number_of_genes"]

print("Loading nt data...")

feature_type = "nt_1mer"
training_data_nt = load_training_data(feature_type=feature_type)
validation_data_nt = load_validation_data(feature_type=feature_type)

print("Calculating GC content...")
train_gc = calc_gc(training_data_nt["features"])
test_gc = calc_gc(validation_data_nt["features"])

print("Loading cds data...")
feature_type = "cds_1mer"
training_data_cds_nt = load_training_data(feature_type=feature_type)
validation_data_cds_nt = load_validation_data(feature_type=feature_type)

train_rna_zc = calc_rna_zc(training_data_cds_nt["features"])
test_rna_zc = calc_rna_zc(validation_data_cds_nt["features"])

# Strip the .x suffixes from the test index
test_rna_zc.index = [a for a,b in test_rna_zc.index.str.split('.')]

print("Loading aa data...")
feature_type = 'aa_1mer'
training_data_aa = load_training_data(feature_type=feature_type)
validation_data_aa = load_validation_data(feature_type=feature_type)

print("Calculating protein ZC...")
train_aa_zc = calc_aa_zc(training_data_aa["features"])
test_aa_zc = calc_aa_zc(validation_data_aa["features"])

print("Merging features...")
training_features = pd.merge(train_gc, train_aa_zc, left_index=True, right_index=True)
validation_features = pd.merge(test_gc, test_aa_zc, left_index=True, right_index=True)
training_features = pd.merge(training_features, train_rna_zc, left_index=True, right_index=True)
validation_features = pd.merge(validation_features, test_rna_zc, left_index=True, right_index=True)
training_features = pd.merge(training_features, train_ngenes, left_index=True, right_index=True)
validation_features = pd.merge(validation_features, test_ngenes, left_index=True, right_index=True)

if training_features['gc_content'].size == 0:
    print("No training features found!")
if validation_features['gc_content'].size == 0:
    print("No validation features found!")

training_data = training_data_aa
validation_data = validation_data_aa
training_data["features"] = training_features
validation_data["features"] = validation_features

cleaned_data = process_data(
    training_data["features"], training_data["labels"]["physiology"],
    validation_data["features"], validation_data["labels"]["physiology"])

Loading metadata...
Loading nt data...
Calculating GC content...
Loading cds data...
Loading aa data...
Calculating protein ZC...
Merging features...


In [4]:
training_data["features"]

,gc_content,cds_aa_zc,cds_aa_NC,cds_aa_NN,cds_aa_NO,cds_aa_NS,cds_nt_zc,cds_nt_NC,cds_nt_NN,cds_nt_NO,cds_nt_NS,number_of_genes
GCA_000007325.1,0.271516,-0.223904,5.143151,1.312394,2.503998,0.031280,0.906262,9.580959,3.843651,8.011308,0.0,2022
GCA_000008085.1,0.315595,-0.261506,5.313145,1.337274,2.473235,0.025246,0.908078,9.579899,3.869975,8.023750,0.0,583
GCA_000009845.1,0.277599,-0.225050,5.268872,1.365388,2.493303,0.028975,0.881118,9.527548,3.735673,8.080170,0.0,956
GCA_000010085.1,0.545049,-0.147696,4.929883,1.381895,2.430102,0.040638,0.908349,9.511738,3.802405,8.265712,0.0,5742
GCA_000010565.1,0.529611,-0.179736,4.949605,1.357630,2.428669,0.036650,0.917596,9.536937,3.859736,8.220832,0.0,3001
...,...,...,...,...,...,...,...,...,...,...,...,...
GCF_904846235.1,0.427827,-0.150589,4.925863,1.330677,2.485576,0.036234,0.899661,9.519435,3.768315,8.197186,0.0,2708
GCF_904846285.1,0.424173,-0.152190,4.936632,1.334622,2.483212,0.035871,0.899430,9.518972,3.765110,8.197720,0.0,2748
GCF_904848585.1,0.619125,-0.136393,4.828909,1.365579,2.412853,0.033587,0.912379,9.505566,3.824078,8.306658,0.0,7761
GCF_905187535.1,0.397108,-0.184541,5.055416,1.320646,2.502971,0.028774,0.898044,9.526067,3.764989,8.166035,0.0,1941


In [5]:
print("Fitting model...")
model = LogisticClassifier(max_iter=100000, normalize=True, random_state=None)
model.fit(cleaned_data["X_train"], cleaned_data["y_train"])
accuracy = model.score(cleaned_data["X_train"], cleaned_data["y_train"])
balanced_accuracy = model.balanced_accuracy(cleaned_data["X_train"], cleaned_data["y_train"])

print("Accuracy: " + str(accuracy))
print("Balanced Accuracy: " + str(balanced_accuracy))

# compute accuracy and balanced accuracy on test set
test_accuracy = model.score(cleaned_data["X_test"], cleaned_data["y_test"])
test_balanced_accuracy = model.balanced_accuracy(cleaned_data["X_test"], cleaned_data["y_test"])
print("Test Accuracy: " + str(test_accuracy))
print("Test Balanced Accuracy: " + str(test_balanced_accuracy))

C = model.confusion_matrix(cleaned_data["X_train"], cleaned_data["y_train"])
C = pd.DataFrame(C,index=model.classifier.classes_, columns=model.classifier.classes_)
C.apply(lambda x: x/x.sum(), axis=1)


Fitting model...
Accuracy: 0.7570505287896592
Balanced Accuracy: 0.580434883626373
Test Accuracy: 0.6106870229007634
Test Balanced Accuracy: 0.5331186258722491


,Aerobe,Anaerobe,Facultative
Aerobe,0.887292,0.108683,0.004025
Anaerobe,0.153972,0.842752,0.003276
Facultative,0.720721,0.268018,0.011261


In [6]:
# Make it a two-class thing and run again
training_data["labels"]["physiology"] = training_data["labels"]["physiology"].replace("Facultative", "Aerobe")
validation_data["labels"]["physiology"] = validation_data["labels"]["physiology"].replace("Facultative", "Aerobe")

cleaned_data = process_data(
    training_data["features"], training_data["labels"]["physiology"],
    validation_data["features"], validation_data["labels"]["physiology"])


print("Fitting two-class model...")
model = LogisticClassifier(max_iter=100000, normalize=True, random_state=None)
model.fit(cleaned_data["X_train"], cleaned_data["y_train"])
accuracy = model.score(cleaned_data["X_train"], cleaned_data["y_train"])
balanced_accuracy = model.balanced_accuracy(cleaned_data["X_train"], cleaned_data["y_train"])

print("Accuracy: " + str(accuracy))
print("Balanced Accuracy: " + str(balanced_accuracy))

# compute accuracy and balanced accuracy on test set
test_accuracy = model.score(cleaned_data["X_test"], cleaned_data["y_test"])
test_balanced_accuracy = model.balanced_accuracy(cleaned_data["X_test"], cleaned_data["y_test"])
print("Test Accuracy: " + str(test_accuracy))
print("Test Balanced Accuracy: " + str(test_balanced_accuracy))

C = model.confusion_matrix(cleaned_data["X_train"], cleaned_data["y_train"])
C = pd.DataFrame(C,index=model.classifier.classes_, columns=model.classifier.classes_)
C.apply(lambda x: x/x.sum(), axis=1)

Fitting two-class model...
Accuracy: 0.8572267920094007
Balanced Accuracy: 0.8390593983814323
Test Accuracy: 0.7633587786259542
Test Balanced Accuracy: 0.768293404318249


,Aerobe,Anaerobe
Aerobe,0.903344,0.096656
Anaerobe,0.225225,0.774775
